In [ ]:
#| default_exp create_umap_layout

In [ ]:
#| export

import numpy as np
import umap
import daft
from bedmap.step import step

# Create umap layout(s)

In [ ]:
#| export

def umap_2d(embeddings: np.ndarray, n_neighbors: int = 15, min_dist: float = 0.1,
                metric: str = "cosine", random_state: int = 42) -> np.ndarray:
    """
    Create a UMAP layout from embeddings.

    Args:
        embeddings: np.ndarray, shape (n_samples, n_features)
        n_neighbors: int, default=15
        min_dist: float, default=0.1
        n_components: int, default=2
    """
    return umap.UMAP(n_neighbors=n_neighbors, min_dist=min_dist,
                     metric=metric, random_state=random_state).fit_transform(embeddings)

In [ ]:
#| hide

umap_2d(np.random.rand(5, 10), n_neighbors=15, min_dist=0.1, metric="cosine", random_state=42)

/home/willsa/git/bedmap-dev/.venv/lib/python3.12/site-packages/umap/umap_.py:2344: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(


array([[ 3.712369 , 14.240169 ],
       [ 4.2498384, 15.165897 ],
       [ 4.7774396, 13.95076  ],
       [ 5.125871 , 14.955444 ],
       [ 4.0246334, 13.347903 ]], dtype=float32)

In [ ]:
#| export

@step(requires=["embeddings"], provides=["umap_xys"])
def create_umap_col(df: daft.DataFrame, n_neighbors: int = 15, min_dist: float = 0.1,
                metric: str = "cosine", random_state: int = 42,
                col_namespace="umap") -> daft.DataFrame:
    """
    Create a UMAP layout from embeddings.
    """
    print("Creating umap layout")
    ## TODO: use namespacing for XYs column
    embeds = df.select("embeddings").to_arrow()["embeddings"]
    shape = len(embeds), embeds.type.list_size
    embeds = np.stack(embeds.combine_chunks().to_numpy(zero_copy_only=False), 0)
    umap_xys = umap_2d(embeds, n_neighbors=n_neighbors, min_dist=min_dist,
                       metric=metric, random_state=random_state)
    umap_xys = daft.Series.from_numpy(umap_xys)
    return df.with_column(f"{col_namespace}_xys", daft.lit(umap_xys))

In [ ]:
#| hide

from bedmap.config import Cfg
import pyarrow as pa


cfg = Cfg()

shape = (5,10)
a = np.random.rand(*shape)
arr = pa.FixedSizeListArray.from_arrays(a.ravel(), shape[-1])

embeds = daft.Series.from_arrow(arr)
df = daft.from_pydict({"embeddings":embeds})

create_umap_col(df, **cfg.umap_spec.model_dump())

Creating umap layout


/tmp/ipykernel_56511/2679538681.py:16: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  create_umap_col(df, **cfg.umap_spec.dict())


🗡️ 🐟 InMemorySource: 00:00 

/home/willsa/git/bedmap-dev/.venv/lib/python3.12/site-packages/umap/umap_.py:2344: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(


embeddingsFixedSizeList[Float64; 10],umap_xysList[List[Float32]]


In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()